<a href="https://colab.research.google.com/github/sonphantrung/Mangio-RVC-Tweaks/blob/pr-optimization/notebooks/Easy_GUI_Testing_Colab_(Experimental).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# THIS IS THE EXPERIMENTAL COLAB FOR TESTERS

##Step 1. Install (it will take 30-45 seconds)

In [ ]:
#@title GPU Check
!nvidia-smi

In [ ]:
#@title Install Dependencies (and load your cached install if it exists to boost times)
# Required Libraries
import os
import csv
import shutil
import tarfile
import subprocess
from pathlib import Path
from datetime import datetime

#@markdown This will forcefully update dependencies even after the initial install seemed to have functioned.
ForceUpdateDependencies = False #@param{type:"boolean"}
#@markdown This will force temporary storage to be used, so it will download dependencies every time instead of on Drive. Not needed, unless you really want that 160mb storage. (Turned on by default for non-training colab to boost the initial launch speed)
ForceTemporaryStorage = True #@param{type:"boolean"}

# Cloudflared
print(f'⚓ Installing cloudflared')
!wget -cq -O /content/cloudflared-linux-amd64.deb "https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb"
!dpkg -i /content/cloudflared-linux-amd64.deb
!rm /content/cloudflared-linux-amd64.deb

!pip install -q gTTS
!pip install -q elevenlabs
!pip install -q stftpitchshift==1.5.1

# Mounting Google Drive
if not ForceTemporaryStorage:
    from google.colab import drive

    if not os.path.exists('/content/drive'):
        drive.mount('/content/drive')
    else:
        print('Drive is already mounted. Proceeding...')

# Function to install dependencies with progress
def install_packages():
    packages = ['build-essential', 'python3-dev', 'ffmpeg', 'aria2']
    pip_packages = ['pip', 'setuptools', 'wheel', 'httpx==0.23.0', 'faiss-gpu', "fastapi==0.88.0", 'fairseq', 'gradio==3.34.0'
                    'ffmpeg', 'ffmpeg-python', 'praat-parselmouth', 'pyworld', 'numpy==1.23.5',
                    'numba==0.56.4', 'librosa==0.9.2', 'mega.py', 'gdown', 'onnxruntime']

    print("Updating and installing system packages...")
    for package in packages:
        print(f"Installing {package}...")
        subprocess.check_call(['apt-get', 'install', '-qq', '-y', package])

    print("Updating and installing pip packages...")
    subprocess.check_call(['pip', 'install', '--upgrade'] + pip_packages)

    print('Packages up to date.')

# Function to scan a directory and writes filenames and timestamps
def scan_and_write(base_path, output_file):
    with open(output_file, 'w', newline='') as f:
        writer = csv.writer(f)
        for dirpath, dirs, files in os.walk(base_path):
            for filename in files:
                fname = os.path.join(dirpath, filename)
                try:
                    mtime = os.path.getmtime(fname)
                    writer.writerow([fname, mtime])
                except Exception as e:
                    print(f'Skipping irrelevant nonexistent file {fname}: {str(e)}')
    print(f'Finished recording filesystem timestamps to {output_file}.')

# Function to compare files
def compare_files(old_file, new_file):
    old_files = {}
    new_files = {}

    with open(old_file, 'r') as f:
        reader = csv.reader(f)
        old_files = {rows[0]:rows[1] for rows in reader}

    with open(new_file, 'r') as f:
        reader = csv.reader(f)
        new_files = {rows[0]:rows[1] for rows in reader}

    removed_files = old_files.keys() - new_files.keys()
    added_files = new_files.keys() - old_files.keys()
    unchanged_files = old_files.keys() & new_files.keys()

    changed_files = {f for f in unchanged_files if old_files[f] != new_files[f]}

    for file in removed_files:
        print(f'File has been removed: {file}')

    for file in changed_files:
        print(f'File has been updated: {file}')

    return list(added_files) + list(changed_files)

# Check if CachedRVC.tar.gz exists
if ForceTemporaryStorage:
    file_path = '/content/CachedRVC.tar.gz'
else:
    file_path = '/content/drive/MyDrive/RVC_Cached/CachedRVC.tar.gz'

content_file_path = '/content/CachedRVC.tar.gz'
extract_path = '/'

def extract_wav2lip_tar_files():
    !wget https://github.com/sonphantrung/Mangio-RVC-Tweaks/releases/download/wav2lip/wav2lip-cache.tar.gz
    !wget https://github.com/sonphantrung/Mangio-RVC-Tweaks/releases/download/wav2lip/wav2lip-HD.tar.gz

    with tarfile.open('/content/wav2lip-cache.tar.gz', 'r:gz') as tar:
        for member in tar.getmembers():
            target_path = os.path.join('/', member.name)
            try:
                tar.extract(member, '/')
            except:
                pass

    with tarfile.open('/content/wav2lip-HD.tar.gz') as tar:
        tar.extractall('/content')

extract_wav2lip_tar_files()

if not os.path.exists(file_path):
    folder_path = os.path.dirname(file_path)
    os.makedirs(folder_path, exist_ok=True)
    print('No cached dependency install found. Attempting to download GitHub backup..')

    try:
        download_url = "https://github.com/kalomaze/QuickMangioFixes/releases/download/release3/CachedRVC.tar.gz"
        !wget -O $file_path $download_url
        print('Download completed successfully!')
    except Exception as e:
        print('Download failed:', str(e))

        # Delete the failed download file
        if os.path.exists(file_path):
            os.remove(file_path)
        print('Failed download file deleted. Continuing manual backup..')

if Path(file_path).exists():
    if ForceTemporaryStorage:
        print('Finished downloading CachedRVC.tar.gz.')
    else:
        print('CachedRVC.tar.gz found on Google Drive. Proceeding to copy and extract...')

    # Check if ForceTemporaryStorage is True and skip copying if it is
    if ForceTemporaryStorage:
         pass
    else:
        shutil.copy(file_path, content_file_path)

    print('Beginning backup copy operation...')

    with tarfile.open(content_file_path, 'r:gz') as tar:
        for member in tar.getmembers():
            target_path = os.path.join(extract_path, member.name)
            try:
                tar.extract(member, extract_path)
            except Exception as e:
                print('Failed to extract a file (this isn\'t normal)... forcing an update to compensate')
                ForceUpdateDependencies = True
        print(f'Extraction of {content_file_path} to {extract_path} completed.')

    if ForceUpdateDependencies:
        install_packages()
        ForceUpdateDependencies = False
else:
    print('CachedRVC.tar.gz not found. Proceeding to create an index of all current files...')
    scan_and_write('/usr/', '/content/usr_files.csv')

    install_packages()

    scan_and_write('/usr/', '/content/usr_files_new.csv')
    changed_files = compare_files('/content/usr_files.csv', '/content/usr_files_new.csv')

    with tarfile.open('/content/CachedRVC.tar.gz', 'w:gz') as new_tar:
        for file in changed_files:
            new_tar.add(file)
            print(f'Added to tar: {file}')

    os.makedirs('/content/drive/MyDrive/RVC_Cached', exist_ok=True)
    shutil.copy('/content/CachedRVC.tar.gz', '/content/drive/MyDrive/RVC_Cached/CachedRVC.tar.gz')
    print('Updated CachedRVC.tar.gz copied to Google Drive.')
    print('Dependencies fully up to date; future runs should be faster.')

In [ ]:
#@title Clone Github Repository
import os
# Change the current directory to /content/
os.chdir('/content/')

# Changes defaults of the infer-web.py
def edit_file(file_path):
    temp_file_path = "/tmp/temp_file.py"
    changes_made = False
    with open(file_path, "r") as file, open(temp_file_path, "w") as temp_file:
        previous_line = ""
        for line in file:
            new_line = line.replace("value=160", "value=128")
            if new_line != line:
                print("Replaced 'value=160' with 'value=128'")
                changes_made = True
            line = new_line

            new_line = line.replace("crepe hop length: 160", "crepe hop length: 128")
            if new_line != line:
                print("Replaced 'crepe hop length: 160' with 'crepe hop length: 128'")
                changes_made = True
            line = new_line

            new_line = line.replace("value=0.88", "value=0.75")
            if new_line != line:
                print("Replaced 'value=0.88' with 'value=0.75'")
                changes_made = True
            line = new_line

            if "label=i18n(\"输入源音量包络替换输出音量包络融合比例，越靠近1越使用输出包络\")" in previous_line and "value=1," in line:
                new_line = line.replace("value=1,", "value=0.25,")
                if new_line != line:
                    print("Replaced 'value=1,' with 'value=0.25,' based on the condition")
                    changes_made = True
                line = new_line

            if 'choices=["pm", "harvest", "dio", "crepe", "crepe-tiny", "mangio-crepe", "mangio-crepe-tiny"], # Fork Feature. Add Crepe-Tiny' in previous_line:
                if 'value="pm",' in line:
                    new_line = line.replace('value="pm",', 'value="mangio-crepe",')
                    if new_line != line:
                        print("Replaced 'value=\"pm\",' with 'value=\"mangio-crepe\",' based on the condition")
                        changes_made = True
                    line = new_line

            temp_file.write(line)
            previous_line = line

    # After finished, we replace the original file with the temp one
    import shutil
    shutil.move(temp_file_path, file_path)

    if changes_made:
        print("Changes made and file saved successfully.")
    else:
        print("No changes were needed.")
repo_path = f"/content/Mangio-RVC-Tweaks"
if not os.path.exists(repo_path):
    # Clone the latest code from the Mangio621/Mangio-RVC-Fork repository
    !git clone -b ui-redesign-2 https://github.com/sonphantrung/Mangio-RVC-Tweaks.git
    os.chdir('/content/Mangio-RVC-Tweaks')
    !wget https://gist.githubusercontent.com/sonphantrung/fdc674e94ce6fb8b2202a9da4b3b0c7e/raw/d5c1e87f71e74ff57392f18c3c128b8c32d72350/EzMode.py
    os.chdir('/content/')
    edit_file(f"/content/Mangio-RVC-Tweaks/infer-web.py")
    # Make necessary output dirs and example files
    !mkdir -p /content/Mangio-RVC-Tweaks/audios
    !wget https://cdn.discordapp.com/attachments/1042377136905920583/1146619288627712110/someguy.mp3 -O /content/Mangio-RVC-Tweaks/audios/someguy.mp3
    !wget https://cdn.discordapp.com/attachments/1042377136905920583/1146619337323597834/somegirl.mp3 -O /content/Mangio-RVC-Tweaks/audios/somegirl.mp3
else:
    print(f"The repository already exists at {repo_path}. Skipping cloning.")

# Download the credentials file for RVC archive sheet
!mkdir -p /content/Mangio-RVC-Tweaks/stats/
!wget -q https://cdn.discordapp.com/attachments/945486970883285045/1114717554481569802/peppy-generator-388800-07722f17a188.json -O /content/Mangio-RVC-Tweaks/stats/peppy-generator-388800-07722f17a188.json

# Forcefully delete any existing torchcrepe dependency from an earlier run
!rm -rf /Mangio-RVC-Tweaks/torchcrepe

# Download the torchcrepe folder from the maxrmorrison/torchcrepe repository
!git clone https://github.com/maxrmorrison/torchcrepe.git
!mv torchcrepe/torchcrepe Mangio-RVC-Tweaks/
!rm -rf torchcrepe  # Delete the torchcrepe repository folder

# Change the current directory to /content/Mangio-RVC-Tweaks
os.chdir(f'/content/Mangio-RVC-Tweaks')
!mkdir -p pretrained uvr5_weights

In [ ]:
#@title Download the Base Model

!wget https://huggingface.co/sonphantrung/ptrained/resolve/main/rmvpe.pt -P /content/Mangio-RVC-Tweaks/
!wget https://huggingface.co/sonphantrung/ptrained/resolve/main/hubert_base.pt -P /content/Mangio-RVC-Tweaks/

!git clone https://github.com/kalomaze/externalcolabcode.git /content/Mangio-RVC-Tweaks/utils

##Models List:
###You can download from **any** link you have as long as it's RVC. (Mega, Drive, etc.)

Biggest organized voice collection at #voice-models in https://discord.gg/aihub

Model archive spreadsheet, sorted by popularity: https://docs.google.com/spreadsheets/d/1tAUaQrEHYgRsm1Lvrnj14HFHDwJWl0Bd9x0QePewNco/

In [ ]:
#@markdown #Step 2. Download The Model
#@markdown Link the URL path to the model (Mega, Drive, etc.) and start the code

from mega import Mega
import os
import shutil
from urllib.parse import urlparse, parse_qs
import urllib.parse
from google.oauth2.service_account import Credentials
import gspread
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup
import requests
import hashlib

def calculate_md5(file_path):
    hash_md5 = hashlib.md5()
    with open(file_path, "rb") as f:
        for chunk in iter(lambda: f.read(4096), b""):
            hash_md5.update(chunk)
    return hash_md5.hexdigest()

# Initialize gspread
scope = ['https://www.googleapis.com/auth/spreadsheets',
         'https://www.googleapis.com/auth/drive.file',
         'https://www.googleapis.com/auth/drive']

config_path = '/content/Mangio-RVC-Tweaks/stats/peppy-generator-388800-07722f17a188.json'

if os.path.exists(config_path):
    # File exists, proceed with creation of creds and client
    creds = Credentials.from_service_account_file(config_path, scopes=scope)
    client = gspread.authorize(creds)
else:
    # File does not exist, print message and skip creation of creds and client
    print("Sheet credential file missing.")

# Open the Google Sheet (this will write any URLs so I can easily track popular models)
book = client.open("RealtimeRVCStats")
sheet = book.get_worksheet(3)  # get the fourth sheet

def update_sheet(url, filename, filesize, md5_hash, index_version):
    data = sheet.get_all_records()
    df = pd.DataFrame(data)

    if md5_hash in df['MD5 Hash'].values:
        idx = df[df['MD5 Hash'] == md5_hash].index[0]

        # Update download count
        df.loc[idx, 'Download Counter'] = int(df.loc[idx, 'Download Counter']) + 1
        sheet.update_cell(idx+2, df.columns.get_loc('Download Counter') + 1, int(df.loc[idx, 'Download Counter']))

        # Find the next available Alt URL field
        alt_url_cols = [col for col in df.columns if 'Alt URL' in col]
        alt_url_values = [df.loc[idx, col_name] for col_name in alt_url_cols]

        # Check if url is the same as the main URL or any of the Alt URLs
        if url not in alt_url_values and url != df.loc[idx, 'URL']:
            for col_name in alt_url_cols:
                if df.loc[idx, col_name] == '':
                    df.loc[idx, col_name] = url
                    sheet.update_cell(idx+2, df.columns.get_loc(col_name) + 1, url)
                    break
    else:
        # Prepare a new row as a dictionary
        new_row_dict = {'URL': url, 'Download Counter': 1, 'Filename': filename,
                        'Filesize (.pth)': filesize, 'MD5 Hash': md5_hash, 'RVC Version': index_version}

        alt_url_cols = [col for col in df.columns if 'Alt URL' in col]
        for col in alt_url_cols:
            new_row_dict[col] = ''  # Leave the Alt URL fields empty

        # Convert fields other than 'Download Counter' and 'Filesize (.pth)' to string
        new_row_dict = {key: str(value) if key not in ['Download Counter', 'Filesize (.pth)'] else value for key, value in new_row_dict.items()}

        # Append new row to sheet in the same order as existing columns
        ordered_row = [new_row_dict.get(col, '') for col in df.columns]
        sheet.append_row(ordered_row, value_input_option='RAW')

condition1 = False
condition2 = False
already_downloaded = False

# condition1 here is to check if the .index was imported. 2 is for if the .pth was.

!rm -rf /content/unzips/
!rm -rf /content/zips/
!mkdir /content/unzips
!mkdir /content/zips

def sanitize_directory(directory):
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        if os.path.isfile(file_path):
            if filename == ".DS_Store" or filename.startswith("._"):
                os.remove(file_path)
        elif os.path.isdir(file_path):
            sanitize_directory(file_path)

url = 'https://huggingface.co/kalomaze/KanyeV2_Redux/resolve/main/KanyeV2_REDUX_40khz.zip'  #@param {type:"string"}
model_zip = urlparse(url).path.split('/')[-2] + '.zip'
model_zip_path = '/content/zips/' + model_zip

#@markdown This option should only be ticked if you don't want your model listed on the public tracker.
private_model = False #@param{type:"boolean"}

if url != '':
    MODEL = ""  # Initialize MODEL variable
    !mkdir -p /content/Mangio-RVC-Tweaks/logs/$MODEL
    !mkdir -p /content/zips/
    !mkdir -p /content/Mangio-RVC-Tweaks/weights/  # Create the 'weights' directory

    if "drive.google.com" in url:
        !gdown $url --fuzzy -O "$model_zip_path"
    elif "/blob/" in url:
        url = url.replace("blob", "resolve")
        print("Resolved URL:", url)  # Print the resolved URL
        !wget "$url" -O "$model_zip_path"
    elif "mega.nz" in url:
        m = Mega()
        print("Starting download from MEGA....")
        m.download_url(url, '/content/zips')
    elif "/tree/main" in url:
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        temp_url = ''
        for link in soup.find_all('a', href=True):
            if link['href'].endswith('.zip'):
                temp_url = link['href']
                break
        if temp_url:
            url = temp_url
            print("Updated URL:", url)  # Print the updated URL
            url = url.replace("blob", "resolve")
            print("Resolved URL:", url)  # Print the resolved URL

            if "huggingface.co" not in url:
                url = "https://huggingface.co" + url

            !wget "$url" -O "$model_zip_path"
        else:
            print("No .zip file found on the page.")
            # Handle the case when no .zip file is found
    else:
        !wget "$url" -O "$model_zip_path"

    for filename in os.listdir("/content/zips"):
        if filename.endswith(".zip"):
            zip_file = os.path.join("/content/zips", filename)
            shutil.unpack_archive(zip_file, "/content/unzips", 'zip')

sanitize_directory("/content/unzips")

def find_pth_file(folder):
    for root, dirs, files in os.walk(folder):
        for file in files:
            if file.endswith(".pth"):
                file_name = os.path.splitext(file)[0]
                if file_name.startswith("G_") or file_name.startswith("P_"):
                    config_file = os.path.join(root, "config.json")
                    if os.path.isfile(config_file):
                        print("Outdated .pth detected! This is not compatible with the RVC method. Find the RVC equivalent model!")
                    continue  # Continue searching for a valid file
                file_path = os.path.join(root, file)
                if os.path.getsize(file_path) > 100 * 1024 * 1024:  # Check file size in bytes (100MB)
                    print("Skipping unusable training file:", file)
                    continue  # Continue searching for a valid file
                return file_name
    return None

MODEL = find_pth_file("/content/unzips")
if MODEL is not None:
    print("Found .pth file:", MODEL + ".pth")
else:
    print("Error: Could not find a valid .pth file within the extracted zip.")
    print("If there's an error above this talking about 'Access denied', try one of the Alt URLs in the Google Sheets for this model.")
    MODEL = ""
    global condition3
    condition3 = True

index_path = ""

def find_version_number(index_path):
    if condition2 and not condition1:
        if file_size >= 55180000:
            return 'RVC v2'
        else:
            return 'RVC v1'

    filename = os.path.basename(index_path)

    if filename.endswith("_v2.index"):
        return 'RVC v2'
    elif filename.endswith("_v1.index"):
        return 'RVC v1'
    else:
        if file_size >= 55180000:
            return 'RVC v2'
        else:
            return 'RVC v1'

if MODEL != "":
    # Move model into logs folder
    for root, dirs, files in os.walk('/content/unzips'):
        for file in files:
            file_path = os.path.join(root, file)
            if file.endswith(".index"):
                print("Found index file:", file)
                condition1 = True
                logs_folder = os.path.join('/content/Mangio-RVC-Tweaks/logs', MODEL)
                os.makedirs(logs_folder, exist_ok=True)  # Create the logs folder if it doesn't exist

                # Delete identical .index file if it exists
                if file.endswith(".index"):
                    identical_index_path = os.path.join(logs_folder, file)
                    if os.path.exists(identical_index_path):
                        os.remove(identical_index_path)

                shutil.move(file_path, logs_folder)
                index_path = os.path.join(logs_folder, file)  # Set index_path variable

            elif "G_" not in file and "D_" not in file and file.endswith(".pth"):
                destination_path = f'/content/Mangio-RVC-Tweaks/weights/{MODEL}.pth'
                if os.path.exists(destination_path):
                    print("You already downloaded this model. Re-importing anyways..")
                    already_downloaded = True
                shutil.move(file_path, destination_path)
                condition2 = True
                if already_downloaded is False and os.path.exists(config_path):
                    file_size = os.path.getsize(destination_path) # Get file size
                    md5_hash = calculate_md5(destination_path) # Calculate md5 hash
                    index_version = find_version_number(index_path)  # Get the index version

if condition1 is False:
    logs_folder = os.path.join('/content/Mangio-RVC-Tweaks/logs', MODEL)
    os.makedirs(logs_folder, exist_ok=True)
# this is here so it doesnt crash if the model is missing an index for some reason

if condition2 and not condition1:
    print("Model partially imported! No .index file was found in the model download. The author may have forgotten to add the index file.")
    if already_downloaded is False and os.path.exists(config_path) and not private_model:
        update_sheet(url, MODEL, file_size, md5_hash, index_version)

elif condition1 and condition2:
    print("Model successfully imported!")
    if already_downloaded is False and os.path.exists(config_path) and not private_model:
        update_sheet(url, MODEL, file_size, md5_hash, index_version)

elif condition3:
    pass  # Do nothing when condition3 is true
else:
    print("URL cannot be left empty. If you don't want to download a model now, just skip this step.")

!rm -r /content/unzips/
!rm -r /content/zips/

In [ ]:
#@markdown # Step 3. Upload files (or do it through colab panel instead)
#@markdown Run this cell to upload your vocal files that you want to use, (or zip files containing audio) to your Colab. <br>
#@markdown Alternatively, you can upload from the colab files panel as seen in the video, but this should be more convenient. This method may not work on iOS.
from google.colab import files
from IPython.display import display, Javascript
import os
import shutil
import zipfile
import ipywidgets as widgets

# Create the target directory if it doesn't exist
target_dir = '/content/Mangio-RVC-Tweaks/audios/'
if not os.path.exists(target_dir):
    os.makedirs(target_dir)

uploaded = files.upload()

for fn in uploaded.keys():
    # Check if the uploaded file is a zip file
    if fn.endswith('.zip'):
        # Write the uploaded zip file to the target directory
        zip_path = os.path.join(target_dir, fn)
        with open(zip_path, 'wb') as f:
            f.write(uploaded[fn])

        unzip_dir = os.path.join(target_dir, fn[:-4])  # Remove the .zip extension from the folder name

        # Extract the zip file
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(unzip_dir)

        # Delete the zip file
        if os.path.exists(zip_path):
            os.remove(zip_path)

        print('Zip file "{name}" extracted and removed. Files are in: {folder}'.format(name=fn, folder=unzip_dir))

        # Display copy path buttons for each extracted file
        for extracted_file in os.listdir(unzip_dir):
            extracted_file_path = os.path.join(unzip_dir, extracted_file)
            extracted_file_length = os.path.getsize(extracted_file_path)

            extracted_file_label = widgets.HTML(
                value='Extracted file "{name}" with length {length} bytes'.format(name=extracted_file, length=extracted_file_length)
            )
            display(extracted_file_label)

            extracted_file_path_text = widgets.HTML(
                value='File saved to: <a href="{}" target="_blank">{}</a>'.format(extracted_file_path, extracted_file_path)
            )

            extracted_copy_button = widgets.Button(description='Copy')
            extracted_copy_button_file_path = extracted_file_path  # Make a local copy of the file path

            def copy_to_clipboard(b):
                js_code = '''
                    const el = document.createElement('textarea');
                    el.value = "{path}";
                    el.setAttribute('readonly', '');
                    el.style.position = 'absolute';
                    el.style.left = '-9999px';
                    document.body.appendChild(el);
                    el.select();
                    document.execCommand('copy');
                    document.body.removeChild(el);
                '''
                display(Javascript(js_code.format(path=extracted_copy_button_file_path)))

            extracted_copy_button.on_click(copy_to_clipboard)
            display(widgets.HBox([extracted_file_path_text, extracted_copy_button]))

        continue

    # For non-zip files
    # Save the file to the target directory
    file_path = os.path.join(target_dir, fn)
    with open(file_path, 'wb') as f:
        f.write(uploaded[fn])

    file_length = len(uploaded[fn])
    file_label = widgets.HTML(
        value='User uploaded file "{name}" with length {length} bytes'.format(name=fn, length=file_length)
    )
    display(file_label)

    # Check if the uploaded file is a .pth or .index file
    if fn.endswith('.pth') or fn.endswith('.index'):
        warning_text = widgets.HTML(
            value='<b style="color: red;">Warning:</b> You are uploading a model file in the wrong place. Please ensure it is uploaded to the correct location.'
        )
        display(warning_text)

    # Create a clickable path with copy button
    file_path_text = widgets.HTML(
        value='File saved to: <a href="{}" target="_blank">{}</a>'.format(file_path, file_path)
    )

    copy_button = widgets.Button(description='Copy')
    copy_button_file_path = file_path  # Make a local copy of the file path

    def copy_to_clipboard(b):
        js_code = '''
            const el = document.createElement('textarea');
            el.value = "{path}";
            el.setAttribute('readonly', '');
            el.style.position = 'absolute';
            el.style.left = '-9999px';
            document.body.appendChild(el);
            el.select();
            document.execCommand('copy');
            document.body.removeChild(el);
        '''
        display(Javascript(js_code.format(path=copy_button_file_path)))

    copy_button.on_click(copy_to_clipboard)
    display(widgets.HBox([file_path_text, copy_button]))

# Remove the original uploaded files from /content/
for fn in uploaded.keys():
    if os.path.exists(os.path.join("/content/", fn)):
        os.remove(os.path.join("/content/", fn))

#Step 4. Start the GUI, then open the public URL. It's gonna look like this:
![alt text](https://i.imgur.com/ZjuyG29.png)

In [ ]:
import os
import time
import fileinput
from subprocess import getoutput
import sys
from IPython.utils import capture
from IPython.display import display, HTML, clear_output
%cd /content/Mangio-RVC-Tweaks/

# test
#@markdown Keep this option enabled to use the simplified, easy interface. [FOR TESTING WE ARE NOT USING THIS]
#@markdown <br>Otherwise, it will use the advanced one that you see in the YouTube guide.
ezmode = False #@param{type:"boolean"}
#@markdown You can try using cloudflare as a tunnel instead of gradio.live if you get Connection Errors.
tunnel = "gradio" #@param ["cloudflared", "gradio"]

if ezmode:
  if tunnel == "cloudflared":
    for line in fileinput.FileInput(f'EzMode.py', inplace=True):
      if line.strip() == 'app.queue(concurrency_count=511, max_size=1022).launch(share=True, quiet=True)':
        # replace the line with the edited version
        line = f'        app.queue(concurrency_count=511, max_size=1022).launch(quiet=True)\n'
      sys.stdout.write(line)
    !pkill cloudflared
    time.sleep(4)
    !nohup cloudflared tunnel --url http://localhost:7860 > /content/srv.txt 2>&1 &
    time.sleep(4)
    !grep -o 'https[^[:space:]]*\.trycloudflare.com' /content/srv.txt >/content/srvr.txt
    time.sleep(2)
    srv=getoutput('cat /content/srvr.txt')
    display(HTML('<h1>Your <span style="color:orange;">Cloudflare URL</span> is printed below! Click the link once you see "Running on local URL".</span></h1><br><h2><a href="' + srv + '" target="_blank">' + srv + '</a></h2>'))
    !rm /content/srv.txt /content/srvr.txt
  elif tunnel == "gradio":
    for line in fileinput.FileInput(f'EzMode.py', inplace=True):
      if line.strip() == 'app.queue(concurrency_count=511, max_size=1022).launch(quiet=True)':
        # replace the line with the edited version
        line = f'        app.queue(concurrency_count=511, max_size=1022).launch(share=True, quiet=True)\n'
      sys.stdout.write(line)
  !python3 EzMode.py --colab --pycmd python3
else:
    !python3 infer-web.py --colab --pycmd python3

* For the original RVC GUI, visit: https://github.com/RVC-Project/Mangio-RVC-Tweaks
* If you need to train a model visit: https://colab.research.google.com/drive/1TU-kkQWVf-PLO_hSa2QCMZS1XF5xVHqs?usp=sharing

#**Consider subscribing to my Patreon!**

Benefits include:
- Full on tech support for AI covers in general
- This includes audio mixing and how to train your own models, with any tier.
- Tech support priority is given to the latter tier.

https://patreon.com/kalomaze

Your support would be greatly appreciated! On top of maintaining this colab, I also write and maintain the Google Docs guides, and plan to create a video tutorial for training voices in the future.


##Credits
**Rejekts** - Original colab author. Made easy GUI for RVC<br>
**RVC-Project dev team** - Original RVC software developers <br>
**Mangio621** - Developer of the RVC fork that added crepe support, helped me get it up and running + taught me how to use TensorBoard<br>
**Kalomaze** - Creator of this colab, added autobackup + loader feature, fixed downloader to work with zips that had parentheses + streamlined downloader, added TensorBoard picture, made the doc thats linked, general God amongst men (def not biased 100%)<br>
**grug** - made some ![](https://cdn.discordapp.com/emojis/603381433368707072.webp?size=16&quality=lossless) edits<br>
**sonphantrung** - made some ![](https://cdn.discordapp.com/emojis/1075493518572785755.webp?size=16&quality=lossless) edits necessary to remove annoying errors

---------------

Backup model archive (outdated): https://huggingface.co/QuickWick/Music-AI-Voices/tree/main